In [1]:
!unzip /content/archive.zip

Archive:  /content/archive.zip
replace shoeTypeClassifierDataset/training/boots/image1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [2]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np
import os

In [3]:
#path to dataset
train_dir = r"/content/shoeTypeClassifierDataset/training"
test_dir = r"/content/shoeTypeClassifierDataset/validation"

In [4]:
#set parameters
IMAGE_SIZE = (224,224)
BATCH_SIZE = 32

#train and validation data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True
)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

Found 1199 images belonging to 6 classes.


In [6]:
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 294 images belonging to 6 classes.


In [7]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 300 images belonging to 6 classes.


In [8]:
#load pre trained VGG16 model
base_model = VGG16(weights='imagenet',include_top=False, input_shape=(224,224,3))
base_model.trainable = False # freeze feaature extractor

In [9]:
# create model
model = Sequential([
    base_model,
    Flatten(),
    Dense(128,activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes,activation='softmax')
])



In [10]:
#compile the model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,926,854 (68.39 MB)

 Trainable params: 3,212,166 (12.25 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [12]:
model.fit(
    train_generator,
    epochs=5,
    validation_data=val_generator
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.2112 - loss: 2.7603 - val_accuracy: 0.3163 - val_loss: 1.6674
Epoch 2/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 31s 812ms/step - accuracy: 0.2772 - loss: 1.6423 - val_accuracy: 0.3741 - val_loss: 1.5812
Epoch 3/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 30s 791ms/step - accuracy: 0.3173 - loss: 1.5763 - val_accuracy: 0.4456 - val_loss: 1.4844
Epoch 4/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 31s 807ms/step - accuracy: 0.3088 - loss: 1.5440 - val_accuracy: 0.4014 - val_loss: 1.4772
Epoch 5/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 30s 791ms/step - accuracy: 0.3234 - loss: 1.5237 - val_accuracy: 0.4388 - val_loss: 1.4007


In [13]:
# evaluate on test data
loss,acc = model.evaluate(test_generator)
print(f"\n test accuracy:{acc:4f}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.6687 - loss: 1.0948

 test accuracy:0.553333


In [14]:
# Predict on test data
y_pred_probs = model.predict(test_generator)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = test_generator.classes
labels = list(test_generator.class_indices.keys())

# Classification report
print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=labels))

# Confusion matrix
print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred))

10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 338ms/step

Classification Report:
               precision    recall  f1-score   support

       boots       0.52      0.82      0.64        50
  flip_flops       0.56      0.86      0.68        50
     loafers       0.61      0.70      0.65        50
     sandals       0.00      0.00      0.00        50
    sneakers       0.45      0.10      0.16        50
soccer_shoes       0.55      0.84      0.67        50

    accuracy                           0.55       300
   macro avg       0.45      0.55      0.47       300
weighted avg       0.45      0.55      0.47       300


Confusion Matrix:
 [[41  3  4  0  0  2]
 [ 5 43  0  0  1  1]
 [ 4 10 35  0  0  1]
 [16 10 16  0  3  5]
 [11  8  1  0  5 25]
 [ 2  3  1  0  2 42]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
